# ---
# Unit11: Ambient Noise Tomography

This notebook has the activities of the Course **ProSeisSN**. It deals with time series processing using a passive seismic dataset using [ObsPy](https://docs.obspy.org/).

#### Dependencies: Obspy, Numpy, Matplotlib
#### Reset the Jupyter notebook in order to run it again, press:
***Kernel*** -> ***Restart & Clear Output***
#### The code $\Downarrow$ BELOW $\Downarrow$ runs a notebook with other dependencies

In [ ]:
#------ Import Libraries
import sys
import os
    
#------ Work with the directory structure to include auxiliary codes
print('\n Local directory ==> ', os.getcwd())
print('  - Contents: ', os.listdir(), '\n')

path = os.path.abspath(os.path.join('..'))
if path not in sys.path:
    sys.path.append(path+"/CodePy")

%run ../CodePy/ImpMod.ipynb

#------ Alter default matplotlib rcParams
from matplotlib import rcParams
import matplotlib.dates as dates
# Change the defaults of the runtime configuration settings in the global variable matplotlib.rcParams
plt.rcParams['figure.figsize'] = 9, 5
#plt.rcParams['lines.linewidth'] = 0.5
plt.rcParams["figure.subplot.hspace"] = (.9)
plt.rcParams['figure.dpi'] = 100
#------ Magic commands
%matplotlib inline
%matplotlib widget
#%pylab notebook
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

---
## Read data files from TTB22 as SEG2. Create a new stream from the SEG2 stream
- Read phone positions
- Select and read data

In [ ]:
"""
====================== READ PHONES LOCATIONS ======================
"""
#------ Read the phones cartesian locations
#--- Reads the CSV file with (x, y)m locations
ttb_loc = u.RGloc('../Data/'+'ttb_loc.dat')
#------ Read the phones geographic locations
#--- Reads the CSV file with (lat,lon) in degress locations
ttb_gloc = u.RGloc('../Data/'+'ttb_gloc.dat')
#
#------ Plot gather in cartesian
p.pgather(ttb_loc[:,1], ttb_loc[:,2], ttb_loc[:,0], coord='cartesian')

In [ ]:
"""
====================== READ THE SEISMIC DATA LOCALLY ======================
File hints:
3710 and 3720 -> several events
3740 -> 2 events
3790 -> 1 event (6-9)s
"""
#------ Read the seismic data
ent = str(np.random.choice(np.arange(3700, 3811, 10)))
ent = input(f'   Enter a file number in [3695, 3810], rtn=random:\n') or ent
ent = ent.rstrip().split(' ')
print(f">> Read with data file {ent}")
ent = '../Data/ttb/'+ent[0]+'.dat'
#
#------- Read the data file as a SEG2 object.
st     = read(ent)
#
#------- Print stream information
dummy = float(st[-1].stats.seg2.RECEIVER_LOCATION)
print(f">> Gather acquired on {st[0].stats.starttime}, has {int(st[0].stats.npts)} data points.")
"""
================= Create a new stream from the SEG2 stream ======================
                         Retain a gather copy
"""
#------ Create a new stream from the SEG2 stream.
#       1) Adds coordinates to gather. Stores a copy in gather0
#       2) Gather baricenter = bcenter.
gather, bcenter = u.creastrm(st, ttb_gloc)
gather0 = gather.copy()
#
#--- Phone choice
phone = None


---
## Data processing
- Filter data
- Display the seismogram
### Filter and look at seismogram and to the frequency content

In [ ]:
#
"""
================= Filter data and look at the frequency contents ======================
                    Create a new stream from the SEG2 stream
"""
#
#------- Remove mean and trend + filter the stream
#--- Filter parameters: change them as you wish.
MTparam = [ 1,   1,    'bp',  10.,   40.,   0,    0]
# └─────> [dtr, line, ftype, Fmin, Fmax, taper, gain]
#                                          └─> data will be windowed at trace normalization and spectral whitening
ent = str(MTparam[3]) + ' ' + str(MTparam[4])
ent = input(f'\n Enter filter min and max frequencies (dflt = {MTparam[3]}, {MTparam[4]})') or ent
ent = ent.rstrip().split(' ')
MTparam[3], MTparam[4] = [float(dummy) for dummy in ent]
#
gather = u.otrstr(gather, MTparam)
#
#------- Check frequency contents to accept preprocessing
#--- Pick up a random phone/trace
phone = phone if phone is not None else np.random.randint(1, len(gather)+1)
print(f' Random phone {phone} ')
#--- Go to trace instead of phone: trace = phone -1
phone = phone - 1
#--- Relative time: nummpy array
time = gather[phone].times(type="relative")
#--- Plot Trace+Spectrogram
p.Pspect(time, gather[phone])
#
#------- Once filtering is accepted create a new backup for gather
ent = input(f' Run this cell again (rtn= No, else plot Spectrogram)?: ') or False
if not ent:
    gather0 = gather.copy()
    print(f' A new stream backup was created.')
else:
    gather = gather0.copy()

In [ ]:
"""
====================== Plot Seismogram ======================
"""
gather.plot(type='section',
            scale=1.3, alpha=.7,
            orientation='horizontal')

In [ ]:
"""
====================== Zoom in to select [t0, t1]======================
"""
#------ Zoom in the seismogram

ent = input(f' Enter t0 and t1 to zoom: ')

ent = ent.rstrip().split(' ')
t0 = float(ent[0])
t1 = float(ent[1])
#
dt = gather[0].stats.starttime
gather.plot(type='section',
            scale=1.3, alpha=.7,
            starttime=dt+t0, endtime=dt+t1,
            orientation='horizontal')

### Down-sample the data
- Down-sample the data to number of pints compatible with the upper limit of the bandpass filter
- Reduce computational costs

In [ ]:
#
gather = gather0.copy()
"""
================= Downsample stream by an integer factor ======================
"""
print(f'\n>> Phone {phone+1} has {gather[phone].stats.npts} data points with a sampling rate of {gather[phone].stats.sampling_rate}Hz,')
dummy =  u.divisors(int(gather[phone].stats.sampling_rate), MTparam[4])
print(f'    this sampling rate can be lowered to the following integer values {dummy}Hz')
ent = input(f'\n<< Enter a new sampling rate from the above list:')
ent = float( ent.rstrip().split(' ')[0] )
"""
Decimate (other possiblity is resample)
1) Only every decimation_factor-th sample remains in the trace.
2) Prior to decimation it is applyed a lowpass filter to prevente aliasing artifacts.
3) To abort when
           len(data) % decimation_factor != 0
    set strict_length=True.
"""
#--- // is a floor division = integer floor. Sanity
factor = int(gather[phone].stats.sampling_rate / ent)
if gather[phone].stats.npts % factor != 0: raise ValueError("Decimation factor is not an integer.")
gather.decimate(factor=factor, strict_length=True)
#--- Check on Fmax
MTparam[4] = MTparam[4] if MTparam[4] <= ent else ent
#
print(f'\n>> Phone {phone+1} has now {gather[phone].stats.npts} data points with a new sampling rate of {gather[phone].stats.sampling_rate}Hz.')
print(f'    Resampled data is FIR low-pass filtered to prevent aliasing, with a decimation factor of  {factor}.')
#
#------- Check frequency contents of a trace to accept downsampling
#--- Relative time: nummpy array
time = gather[phone].times(type="relative")
#--- Plot Trace+Spectrogram
p.Pspect(time, gather[phone])
#
#------- Once filtering is accepted create a new backup for gather
ent = input(f' Run this cell again (rtn= No)?: ') or False
if not ent:
    gather0 = gather.copy()
    print(f' A new stream backup was created.')
else:
    gather = gather0.copy()

---
## The direction-of-arrival (DOA). Choose an event with Beamforming
- A wavefront arrives at the surface at an angle $i$ with the vertical. The wave propagates toward the surface with a velocity $v_{c}=\frac{\Delta s}{\varDelta t}$, with a horizontal component $v_{h}=\frac{\Delta x}{\varDelta t}$.

- The horizontal slowness, $u_h$ is the inverse value of horizontal apparent velocity, $1/v_{h}$,
$$
u_{h}=\frac{1}{v_{h}}=\frac{\sin i}{\left|\mathbf{v}_{c}\right|},
$$
being related to: (a) the angle of incidence $i$, (b) the true velocity $v_c$ and (c) its azimuth with the North *toward* the epicenter; the **baz** ($\theta$).
$$
\boldsymbol{U}_0 = (\frac{\sin\theta}{v_{h}},\frac{\cos\theta}{v_{h}},\frac{1}{v_{h}\tan i})
                 = \frac{1}{v_{c}}(\sin i\sin\theta,\sin i\cos\theta,\cos i)
                 = u_{h}(\sin\theta,\cos\theta,\frac{1}{\tan i})
                 = \frac{1}{v_{c}}(\sin i\sin\theta,\sin i\cos\theta,\cos i).
$$

- The seismic signals at each sensor can be time-shifted and summed to enhance the S/N ratio by a factor of $\sqrt{N}$; the signals interfere constructively. The relative time shift of a given sensor $\boldsymbol{r}_{i},\,i=1,\ldots,N$, relatively to the center of the array is 
$$
\tau_{i}=\boldsymbol{r}_{i}.\boldsymbol{u}.
$$

- The beamforming for the array is,
$$
		b\left(t\right)=\frac{1}{N}\mathop{\sum_{i=1}^{N}s_{i}\left(t+
			\mathbf{r}_{i}\mathbf{\cdot u}\right)}=\frac{1}{N}\mathop{\sum_{i=1}^{N}s_{i}\left(t+\tau_{i}\right)}
$$

The ObsPy FK beamforming outputs the relative power, or semblance, and the absolute power, or FK power. Transforms the semblance $S$ to the Fisher or $F$-statistic as $F = (N-1) \frac{S}{1-S}$. $F\rightarrow1$ for white Gaussian noise, therefore if $F\neq1$ means that there is some signal.

<div style="text-align: center;">
<img src="./array1.png" width="600">
</div>



In [ ]:
"""
====================== BEAMFORMING ======================
"""
dummy = UTCDateTime()
# ---------- Use FK Analysis
out, stime, etime = u.BeamFK(gather, [MTparam[3], MTparam[4]], ttb_gloc)
print("\n>> Total time in Beamforming: %f\n" % (UTCDateTime() - dummy))
#---------- Change output
t, rel_power, abs_power, baz, slow = out.T
#--- Time
T = np.linspace(stime, etime, num=len(out))
#--- Semblance -> Fisher
F = (len(gather)-1) * out[:, 1] / (1 - out[:, 1])
#--- FK power
FKp = out[:, 2]
#--- baz
#out[:, 3] = out[:, 3] % 360.
baz = baz % 360.
#--- Slowness -> Velocity
V = 1.e3 / out[:,4]
#------------- print
sys.stdout.write('\n')
print(f'\n>>  t   Fisher  FKpwr   baz(deg) vel(m/s)')    
for i in range(len(out)):
    print(f'   {round(T[i],2)}, {round(F[i],2)}, {round(FKp[i],2)}, {round(baz[i],2)}, {round(V[i],2)}')    
#------------ Plot
p.pltbaz(out, stime , etime)
"""
1) CLICK ON BLUE BAR TO EXPAND
"""

- Is this velocity expected to be real?

<div style="text-align: center;">
<img src="./rockVel.png" width="500">
</div>


### Local functions

In [ ]:
#
# ---------- Pairwise distances between points ----------
"""
  <Args>
    matrix -> A 3-column matrix: [[phone, x, y], ...] (x, y = cartesian coordinates)
    ainc   -> An angle increment
  <Returns>
    maxdist -> A 4-column matrix: [[phone1, phone2, dist,  angle], ...]
                                      int     int   float  float
                    angle -> (degrees) polar angle
                    dist  -> distance between phone1 and phone2)
                    maxdist is filtered by the largest distance near a given angle

"""
def pairs(matrix, angle_increment):
#
#------  Convert to numpy array if necessary
    if not isinstance(matrix, np.ndarray):
        matrix = np.array(matrix)
#
#------  Calculate the baricenter
    baricenter_x = np.mean(matrix[:, 1])
    baricenter_y = np.mean(matrix[:, 2])
#
#------ Create an empty list to store results
    results = []
#
#------ Iterate through angles 
    for angle in range(0, 180, angle_increment):
      # Convert angle to radians
        rad = np.radians(angle)
        # Define the diameter
        diameter_slope = np.tan(rad)
#
#------  Find the pair of points with minimum distance from the current diameter
        min_distance = float('inf')
        point1_index = -1
        point2_index = -1
        for i in range(len(matrix)):
            for j in range(i + 1, len(matrix)):
                # Calculate distance from the line
                dist_i = abs((matrix[i, 2] - baricenter_y) - diameter_slope * (matrix[i, 1] - baricenter_x)) / (np.sqrt(1 + diameter_slope**2))
                dist_j = abs((matrix[j, 2] - baricenter_y) - diameter_slope * (matrix[j, 1] - baricenter_x)) / (np.sqrt(1 + diameter_slope**2))
                total_dist = dist_i + dist_j

                if total_dist < min_distance:
                    
                   # print(point1_index,point2_index,total_dist,min_distance)
                    
                    min_distance = total_dist
                    point1_index = int(matrix[i,0])
                    point2_index = int(matrix[j,0])
#
#------ Calculate distance between the selected pair
        distance = np.sqrt((matrix[int(np.where(matrix[:,0] == point1_index)[0]), 1] - matrix[int(np.where(matrix[:,0] == point2_index)[0]), 1])**2 + (matrix[int(np.where(matrix[:,0] == point1_index)[0]), 2] - matrix[int(np.where(matrix[:,0] == point2_index)[0]), 2])**2)
#
#------ Angle relative to vertical axis
        results.append([int(point1_index), int(point2_index), distance, angle])  #90. - angle
    return np.array(results, dtype=object)
#
# -------------- End of function   ---------------------
print(f'\n>> Functions loaded.')

---
### Choose phone pairs

<div style="text-align: center;">
<img src="./ttbg.png" width="500">
</div>


In [ ]:
"""
====================== Phone pairs ======================
"""
print(f'\n>> Choose pairs of the {len(gather)} using the polar angle of a rotating diameter.')
ent = input(f'\n<< Enter an angle step (rtn = 20 deg.):') or '20'
ent = int( ent.rstrip().split(' ')[0] )
matrix = pairs(ttb_loc, ent)
print(f'ph1|ph2|     distance(m)    |angle(deg.)|')
for row in matrix:
    print(" | ".join(map(str, row)))
#
#------ Choose a single pair
ent = input(f'\n<< Enter the 1st phone from above table:')
ent = int( ent.rstrip().split(' ')[0] )
i = next((i for i, row in enumerate(matrix) if row[0] == ent), -1)
print(f'\n>> Work with the pair [{matrix[i,0]}, {matrix[i,1]}], with a distance of {round(matrix[i,2],2)}m')
#
#------ Create an empty stream add the two traces
distance = matrix[i,2]
ntr1, ntr2 = matrix[i,:2] - 1
st = Stream()
st += gather[ntr1].copy()
st += gather[ntr2].copy()
print(st)

### Normalization and Spectral whitening

In [ ]:
"""
====================== Trace normalization and spectral whitening ======================
"""
dt   = st[0].stats.delta               # sampling interval
fNy  = 1. / (2.0 * dt)                    # Nyquist frequency
#- time = st[ntr1].times(type="relative")
#
#------ Normalize and whiten
for tr in st:
#--- Taper the data with 10% Hanning
    tr.taper(type = 'hann', max_percentage = 0.1)
#--- Time normalization == 'one_bit' -> sign normalization
    tr = np.sign(tr)
#--- Whiten
#    tr = u.whiten(tr, MTparam[3], MTparam[4])
#    trw = whiten(tr, delta = dt, freqmin = MTparam[3], freqmax = MTparam[4], smooth_N  = 100)

## Ambient Noise Cross-correlation
- Given two seismometers, $u_1$ and $u_2$, on the surface, will record ground motion as a function of time. Over long periods of time, the cross-correlation of ground motions is
$$C_{1,2}\left(\tau\right)=\int u_{1}\left(t\right)\,u_{2}\left(t+\tau\right)dt$$

- Data Preparation and inital processing
Prepare waveform data from each station separately to accentuate broad-band ambient noise.

In [ ]:
"""
====================== Trace correlation ======================
"""
print(f'\n>> Correlate trace {ntr1} with {st[0].stats.npts} with trace {ntr2} with {st[1].stats.npts} points')
ent = input(f'\n<< Enter max lag time (rtn = 2s):') or '2'
ent = float( ent.rstrip().split(' ')[0] )
max_lagtime = ent
#
max_shift_num = int(np.round(max_lagtime*st[0].stats.sampling_rate))
data1 = st[0].data
data2 = st[1].data
len1 = len(data1)
len2 = len(data2)
min_len = min(len1,len2)
#
cross_list = []
for shift_num in np.arange(-max_shift_num,max_shift_num+1,1):
    if shift_num<0:
        correlate_value = np.correlate(data1[:min_len+shift_num],data2[-shift_num:min_len])
        cross_list.append(correlate_value.ravel())
    else:
        correlate_value = np.correlate(data2[:min_len-shift_num],data1[shift_num:min_len])
        cross_list.append(correlate_value.ravel())
cross_list = np.array(cross_list)
cross_list = cross_list/np.max(cross_list)
#
fs_new = st[0].stats.sampling_rate
time = np.linspace(-max_lagtime,max_lagtime,int(2*max_lagtime*fs_new+1))
#-------- 
indexmax = np.argmax(cross_list)
travtime = time[indexmax]
print(f'\n>> Maximum lag = {travtime}s, corresponding to a velocity {np.round(distance/travtime, 2)}m/s')
#
plt.figure(figsize=(6, 2), dpi=180)
plt.plot(time,cross_list, 'k-')
#plt.plot(time, envelope, 'r-')
plt.axvline(travtime, 0.85, 1, color='b', lw=3)
plt.xlabel("Time (s)")
plt.ylabel("X-cor Coeff")
plt.xlim(-max_lagtime,max_lagtime)
plt.show();